In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 1
def changeBrightness(img, brightness):
    img = np.clip(img + brightness, 0, 255)
    img = Image.fromarray(np.uint8(img))
    return img

# 2
def changeContrast(img, contrast):
    img = np.clip(img * contrast, 0, 255)
    img = Image.fromarray(np.uint8(img))
    return img

# 3
def flip(img, direction):
    imgTemp = img.copy()
    if direction == 0: # vertical
        numRow = img.shape[0]
        for i in range(numRow):
            img[i] = imgTemp[numRow - i - 1]
    elif direction == 1: # horizontal
        numCol = img.shape[1]
        for i in range(numCol):
            img[:, i] = imgTemp[:, numCol - i - 1]

    img = Image.fromarray(np.uint8(img))
    return img

# 4
def convertRGB(img, type):
    if type == 'Grayscale':
        img = np.clip(np.dot(img, [0.299, 0.587, 0.114]), 0, 255)
    elif type == 'Sepia':
        for row in img:
            for pixel in row:
                tmp = np.copy(pixel)
                pixel[0] = min(255, tmp[0] * 0.393 + tmp[1] * 0.769 + tmp[2] * 0.189)
                pixel[1] = min(255, tmp[0] * 0.349 + tmp[1] * 0.686 + tmp[2] * 0.168)
                pixel[2] = min(255, tmp[0] * 0.272 + tmp[1] * 0.534 + tmp[2] * 0.131)

    img = Image.fromarray(np.uint8(img))
    return img

# 5
def convolve(img, kernel, channel):
    imgOut = np.zeros((img.shape[0], img.shape[1]))

    pad = (kernel.shape[0] - 1) // 2
    imgTemp = np.zeros((img.shape[0] + 2*pad, img.shape[1] + 2*pad))
    imgTemp[pad:imgTemp.shape[0] - pad, pad:imgTemp.shape[1] - pad] = img[:,:,channel].reshape((-1, img.shape[1]))
 
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            imgOut[row, col] = np.sum(kernel * imgTemp[row:row + kernel.shape[0], col:col + kernel.shape[1]])

    return imgOut

def convolution(image, kernel):
    newImg = np.dstack((convolve(image, kernel, 0), convolve(image, kernel, 1), convolve(image, kernel, 2)))
    return newImg

def dnorm(x, sigma):
    return np.array(1 / (np.sqrt(2 * np.pi) * sigma) * (np.exp(-np.power(x / sigma, 2) / 2)))

def generateKernel(size, sigma=1):
    kernel1d = np.linspace(-(size // 2), size // 2, num=size)
    kernel1d = dnorm(kernel1d, sigma)
    kernel2d = np.outer(kernel1d.T, kernel1d.T)
    kernel2d *= 1.0 / np.sum(kernel2d)
    return kernel2d

def blur(img, kernelSize):
    kernel = generateKernel(kernelSize, sigma=np.sqrt(kernelSize))
    img = convolution(img, kernel)
    return Image.fromarray(np.uint8(img))

def sharpen(img):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    img = convolution(img, kernel)
    img = np.clip(img, 0, 255)
    return Image.fromarray(np.uint8(img))

# 6
def cropBySize(img, height, width):
    center = (img.shape[0]//2, img.shape[1]//2)
    imgOut = img[center[0]-height//2 : center[0]+(height-height//2), center[1]-width//2 : center[1]+(width-width//2)]
    return Image.fromarray(np.uint8(imgOut))

def generateCircleMask(img):
    distances = np.zeros((img.shape[0], img.shape[1]))

    center = (img.shape[1]//2, img.shape[0]//2)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            distances[i][j] = (j - center[0])**2 + (i - center[1])**2
    distances = np.sqrt(distances)

    radius = min(center[0], center[1])
    return (distances <= radius)

# 7
def cropByCircle(img):
    mask = generateCircleMask(img)
    mask = np.dstack([mask]*3)
    img *= mask
    return Image.fromarray(np.uint8(img))

# main
if __name__=="__main__":
    imgPath = input("Image path: ")
    img = Image.open(imgPath)
    imgName = img.filename.split('.')[0]
    img = np.array(img)

    img = img.astype(np.int16)

    print("0. All functions")
    print("1. Change the brightness")
    print("2. Change the contrast")
    print("3. Flip")
    print("4. Convert RGB to grayscale/sepia")
    print("5. Blur/Sharpen")
    print("6. Crop by size")
    print("7. Crop by a circle")

    choice = int(input("Choice: "))

    match choice:
        case 0:
            brightness = int(input("Brightness (-255 to 255): "))
            imgOut = changeBrightness(img, brightness)
            imgOut.save(imgName + "_brightness.png")
            plt.imshow(imgOut)
            plt.show()

            contrast = float(input("Contrast (0<a<1: low contrast, a>1: high contrast): "))
            imgOut = changeContrast(img, contrast)
            imgOut.save(imgName + "_contrast.png")
            plt.imshow(imgOut)
            plt.show()

            imgOut = flip(img, 0)
            imgOut.save(imgName + "_flip_vertical.png")
            plt.imshow(imgOut)
            plt.show()
            imgOut = flip(img, 1)
            imgOut.save(imgName + "_flip_horizontal.png")
            plt.imshow(imgOut)
            plt.show()

            imgOut = convertRGB(img, 'Grayscale')
            imgOut.save(imgName + "_grayscale.png")
            plt.imshow(imgOut)
            plt.show()
            imgOut = convertRGB(img, 'Sepia')
            imgOut.save(imgName + "_sepia.png")
            plt.imshow(imgOut)
            plt.show()

            imgOut = blur(img, 15)
            imgOut.save(imgName + "_blur.png")
            plt.imshow(imgOut)
            plt.show()
            imgOut = sharpen(img)
            imgOut.save(imgName + "_sharpen.png")
            plt.imshow(imgOut)
            plt.show()

            height = int(input("Input height: "))
            width = int(input("Input width: "))
            imgOut = cropBySize(img, height, width)
            imgOut.save(imgName + "_cropbysize.png")
            plt.imshow(imgOut)
            plt.show()

            imgOut = cropByCircle(img)
            imgOut.save(imgName + "_cropbycircle.png")
            plt.imshow(imgOut)
            plt.show()

        case 1:
            brightness = int(input("Brightness (-255 to 255): "))
            img = changeBrightness(img, brightness)
            img.save(imgName + "_brightness.png")
            plt.imshow(img)
            plt.show()
        case 2:
            contrast = float(input("Contrast (0<a<1: low contrast, a>1: high contrast): "))
            img = changeContrast(img, contrast)
            img.save(imgName + "_contrast.png")
            plt.imshow(img)
            plt.show()
        case 3:
            print("0. Vertical")
            print("1. Horizontal")
            direction = int(input("Choice: "))
            if direction == 0:
                img = flip(img, direction)
                img.save(imgName + "_flip_vertical.png")
            elif direction == 1:
                img = flip(img, direction)
                img.save(imgName + "_flip_horizontal.png")
            plt.imshow(img)
            plt.show()
        case 4:
            print("0. Grayscale")
            print("1. Sepia")
            type = int(input("Choice: "))
            if type == 0:
                img = convertRGB(img, 'Grayscale')
                img.save(imgName + "_grayscale.png")
            elif type == 1:
                img = convertRGB(img, 'Sepia')
                img.save(imgName + "_sepia.png")
            plt.imshow(img)
            plt.show()
        case 5:
            print("0. Blur")
            print("1. Sharpen")
            type = int(input("Choice: "))
            if type == 0:
                img = blur(img, 15)
                img.save(imgName + "_blur.png")
            elif type == 1:
                img = sharpen(img)
                img.save(imgName + "_sharpen.png")
            plt.imshow(img)
            plt.show()
        case 6:
            height = int(input("Input height: "))
            width = int(input("Input width: "))
            img = cropBySize(img, height, width)
            img.save(imgName + "_cropbysize.png")
            plt.imshow(img)
            plt.show()
        case 7:
            img = cropByCircle(img)
            img.save(imgName + "_cropbycircle.png")
            plt.imshow(img)
            plt.show()